Fase 3: Data Preparation

Objetivo de la fase: El objetivo de la preparación de datos es crear un conjunto de datos limpio y adecuado que permita implementar el modelo de recomendación de libros. Esto incluye la selección de las variables relevantes, el manejo de los datos faltantes, la codificación de variables categóricas, y la transformación de los datos en un formato que pueda ser utilizado en el análisis.

In [1]:
# Imports

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import kaggle
import os

In [2]:
csv_file_path = 'data/books.csv'

In [3]:
if not os.path.exists(csv_file_path):
    kaggle.api.dataset_download_files('jealousleopard/goodreadsbooks', path='data/', unzip=True)

In [4]:
df = pd.read_csv(csv_file_path, delimiter=',', on_bad_lines='skip')

In [5]:
# Creamos un nuevo DataFrame para trabajar
df1 = df.copy()

In [6]:
# Eliminar espacios extra del nombre de la columna num_pages
df1.rename(columns={'  num_pages': 'num_pages'}, inplace=True)

In [7]:
# Revisar los valores faltantes en el dataset
missing_values = df1.isnull().sum()
print(missing_values)

# Eliminar filas con valores faltantes
df1_clean = df1.dropna()

# O, si es más conveniente, rellenar los valores faltantes
df1_clean = df1.fillna(method='ffill')  # Forward fill

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
num_pages             0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64


C:\Users\reyes\AppData\Local\Temp\ipykernel_7956\3286960657.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df1_clean = df1.fillna(method='ffill')  # Forward fill


In [8]:
# Normalizamos las variables numéricas
scaler = MinMaxScaler()
df1[['average_rating', 'ratings_count', 'text_reviews_count', 'num_pages']] = scaler.fit_transform(
    df1[['average_rating', 'ratings_count', 'text_reviews_count', 'num_pages']]
)

In [9]:
# Se crea la matriz de variables numéricas
features = ['average_rating', 'ratings_count', 'text_reviews_count', 'num_pages']
X = df1[features]

In [10]:
# Convertir la columna 'publication_date' a formato de fecha
df1_clean['publication_date'] = pd.to_datetime(df1_clean['publication_date'], errors='coerce')

In [11]:
#Agrupamos valores numericos en categorias
bins = [0, 2.5, 4.0, 5.0]
labels = ['Bajo', 'Medio', 'Alto']
df1['rating_category'] = pd.cut(df1['average_rating'], bins=bins, labels=labels)

Eliminación de outliers

In [12]:
# Establecemos que variables son númericas
numerical_columns = ['average_rating', 'ratings_count', 'text_reviews_count', 'num_pages']

# Calculamos el rango intercuartílico e identificamos los outliers.
def detect_outliers_iqr(df1, column):
    Q1 = df1[column].quantile(0.25)
    Q3 = df1[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df1[(df1[column] < lower_bound) | (df1[column] > upper_bound)]
    
    return outliers

In [13]:
# Detectamos outliers en las variables númericas
for column in numerical_columns:
    outliers = detect_outliers_iqr(df1, column)
    print(f'Outliers en {column}:')
    print(outliers)

# Eliminamos los outliers
def remove_outliers(df, column):
    Q1 = df1[column].quantile(0.25)
    Q3 = df1[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtramos el DataFrame eliminando los outliers
    df1_filtered = df1[(df1[column] >= lower_bound) & (df1[column] <= upper_bound)]
    
    return df1_filtered

# Para cada columna númerica le eliminamos los outliers correspondientes.
for column in numerical_columns:
    df1 = remove_outliers(df1, column)

Outliers en average_rating:
       bookID                                              title  \
4           8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...   
6          10       Harry Potter Collection (Harry Potter  #1-6)   
96        159                          Dinner with Anna Karenina   
247       681  Natural Cures "They" Don't Want You to Know about   
265       799  Out to Eat London 2002 (Lonely Planet Out to Eat)   
...       ...                                                ...   
10985   45052                         The Mini Zen Gardening Kit   
10986   45054  The Zen Gardening Kit/Book and Japanese Rock G...   
11027   45259  Anthony Thwaite: In Conversation With Peter Da...   
11070   45444                                        Going Potty   
11091   45516             The Die Broke Financial Problem Solver   

                                                 authors  average_rating  \
4                             J.K. Rowling/Mary GrandPré           0.956   
6  

In [14]:
# Mostramos el DataFrame sin outliers
print("Data después de eliminar outliers:")
print(df1.describe())

Data después de eliminar outliers:
             bookID  average_rating        isbn13    num_pages  ratings_count  \
count   7810.000000     7810.000000  7.810000e+03  7810.000000    7810.000000   
mean   22047.356978        0.788191  9.755868e+12     0.044005       0.000239   
std    13040.563625        0.053221  4.839071e+11     0.024118       0.000402   
min        4.000000        0.644000  8.987060e+09     0.000000       0.000000   
25%    11208.500000        0.754000  9.780375e+12     0.027828       0.000014   
50%    21933.500000        0.790000  9.780675e+12     0.041363       0.000070   
75%    32767.250000        0.824000  9.780973e+12     0.058394       0.000281   
max    45641.000000        0.938000  9.789999e+12     0.111922       0.002748   

       text_reviews_count  
count         7810.000000  
mean             0.000543  
std              0.000670  
min              0.000000  
25%              0.000064  
50%              0.000255  
75%              0.000796  
max        

Luego de la eliminación de los outliers quedamos con 7.810 datos.

Para finalizar estas etapa podemos quedarnos con: Existe una correlación entre la ratings_count y text_reviews_count, esto se puede significar a que mientras 
calificaciones se hacen, el lector suele hacer reseñas. También las calificaciones promedio van entre 3 a 4.5
lo que indica que los libros están bien valorado por los lectores. Hicimos una copia del dataframe para trabajarlo, eliminamos espacio extra de una columna, normalizamos variables numéricas, creamos matriz de variables numericas, convertimos la columna publication_date a fecha para poder trabajarlo de una mejor manera, agrupamos valores numericos en cateogorias y Al comienzo teniamos 11.123 datos y luego de la eliminación de outliers quedamos en 7.810.

Aplicamos OneHotEncoder en los datos que vamos a trabajar para la tarea de regresión

In [ ]:
# Eliminar columnas irrelevantes
df = df.drop(columns=['bookID', 'isbn', 'isbn13'])

# Manejo de valores nulos (opciones: eliminar filas o rellenar)
df = df.dropna()  # Esto elimina las filas con valores nulos

# Codificar variables categóricas (por ejemplo, autores, idioma, editorial)
from sklearn.preprocessing import OneHotEncoder

# Seleccionar las columnas categóricas
categorical_cols = ['authors', 'language_code', 'publisher']

# Aplicar OneHotEncoder para convertirlas en variables numéricas
encoder = OneHotEncoder(sparse=False)
encoded_df = pd.DataFrame(encoder.fit_transform(df[categorical_cols]))

# Añadir las columnas codificadas de nuevo al dataframe
df = df.drop(columns=categorical_cols).join(encoded_df)

print(df.head())

Preparamos datos para el modelo de regresión

In [ ]:
# 1. Selección de variables relevantes
# Seleccionamos algunas columnas para la predicción, puedes ajustar según tu necesidad
features = ['num_pages', 'ratings_count', 'text_reviews_count', 'authors', 'publisher']
target = 'average_rating'

# Dividir las características (X) y la variable objetivo (y)
X = df1[features]
y = df1[target]

# 2. Manejo de valores nulos
# Imputamos valores faltantes con la mediana para las variables numéricas
# y con el valor más frecuente (modo) para las categóricas.

# Separación de variables numéricas y categóricas
numeric_features = ['num_pages', 'ratings_count', 'text_reviews_count']
categorical_features = ['authors', 'publisher']

# Definir el transformador para las variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputamos los valores nulos con la mediana
    ('scaler', StandardScaler())  # Estandarizamos los datos numéricos
])

# Definir el transformador para las variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputamos los valores nulos con el valor más frecuente
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificación one-hot para las categóricas
])

# 3. Creación del preprocesador (Pipeline)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# 4. Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Aplicar el preprocesamiento
# Ajustamos el preprocesador solo sobre el conjunto de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)

# También transformamos el conjunto de prueba
X_test_processed = preprocessor.transform(X_test)

# Ahora, X_train_processed y X_test_processed están listos para ser usados en el modelado.